# CLOSE AND OPEN EYE CLASSIFIER TRAINING

Once we have extract the eyes image from the original image we can use and train a classifier using a CNN for labeling eyes as open or close, using our dataset.

This first model has 4 convolutional layers, with 16*n kernels of size 3x3, an output layer with a sigmoid activation funcion for binary classification, and Adam optimizer. Also a reduced LR callback has been used every 5 epochs without improvement in the validation accuracy.

This model has 100% accuracy for train dataset, 97% for test dataset, and 95% for another public dataset download from kaggle.

There is a 2nd model with also 4 convolutional layers, but the 1st layer with less kernels applied, and the callback function for reducing LR has been change to 10 epochs without improvement. It has better accuracy for our dataset, but worse for kaggle dataset.

Finally, this models are saved to .h5 files, so they can be load and used in the eye detection code.

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import os
import shutil
import sklearn
import Augmentor as aug
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import time
from IPython.display import Image, display
import matplotlib.cm as cm
from keras.models import load_model

In [2]:
def brightness_augment(img, factor=0.5): 
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
    hsv = np.array(hsv, dtype=np.float64)
    #hsv[:, :, 2] = hsv[:, :, 2] * (factor + np.random.uniform()) #scale channel V uniformly
    hsv[:, :, 2] = hsv[:, :, 2] * (factor) #scale channel V uniformly
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
    rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)
    return rgb

In [3]:
def create_directory(path):
    if os.path.isdir(path) == False:
        try:
            os.makedirs(path)
        except OSError:
            print ("Creation of the directory %s failed" % path)
        else:
            print ("Successfully created the directory %s " % path)

In [4]:
def load_images(root):
    #Method to load all the images and create X data and labels
    data = []
    for category in sorted(os.listdir(root)):
        for file in sorted(os.listdir(os.path.join(root, category))):
            data.append((category, os.path.join(root, category,  file)))

    df = pd.DataFrame(data, columns=['class', 'file_path'])
    df['class'] = LabelEncoder().fit_transform(df['class'].values)
    photos, labels, names = list(), list(), list()
    for index,row in df.iterrows():
        photo = load_img(row['file_path'], color_mode = "grayscale", target_size=(80, 80))
        photo = img_to_array(photo)
        photo = photo.astype("float32") / 255
        photos.append(photo)
        labels.append(row['class'])
        names.append(row['file_path'])
    X = np.asarray(photos)
    y = np.asarray(labels)
    return X, y, names

In [5]:
def change_size(root):
    directory = os.path.join(root,"rescaled")
    create_directory(directory)
    for file in sorted(os.listdir(root)):
        if os.path.isfile(os.path.join(root,file)):
            if file.split(".")[1] in ["jpg","bmp","tiff","png"]:
                path = os.path.join(root,file)
                initial_img = cv2.imread(path)
                gray = cv2.cvtColor(initial_img, cv2.COLOR_BGR2GRAY)
                mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
                img = cv2.inpaint(gray, mask, 3, cv2.INPAINT_TELEA) 
                image = resize(img, (80, 80), preserve_range=True).astype(np.uint8)
                name = os.path.join(directory,file)
                cv2.imwrite(name, image)

In [6]:
class CustomCallbackReduceLr(keras.callbacks.Callback):
    def __init__(self):
        self.val_accuracy = np.Inf
        self.accumulator = 0
    def on_epoch_begin(self, epoch, logs={}):
        if self.accumulator == 5:
            self.model.optimizer.lr = self.model.optimizer.lr*0.5
            self.accumulator = 0
            print("Changing Learning Rate to ", float(keras.backend.get_value(self.model.optimizer.lr)))
    def on_epoch_end(self, epoch, logs={}):
        current = logs["val_accuracy"]
        if (current - self.val_accuracy) < 0.05:
            self.accumulator = self.accumulator + 1
            print("No change on val accuracy, increment accumulator to ", self.accumulator)
        else:
            self.accumulator = 0
        self.val_accuracy = current

In [7]:
class CustomCallbackReduceLr2(keras.callbacks.Callback):
    def __init__(self):
        self.val_accuracy = np.Inf
        self.accumulator = 0
    def on_epoch_begin(self, epoch, logs={}):
        if self.accumulator == 10:
            self.model.optimizer.lr = self.model.optimizer.lr*0.5
            self.accumulator = 0
            print("Changing Learning Rate to ", float(keras.backend.get_value(self.model.optimizer.lr)))
    def on_epoch_end(self, epoch, logs={}):
        current = logs["val_accuracy"]
        if (current - self.val_accuracy) < 0.05:
            self.accumulator = self.accumulator + 1
            print("No change on val accuracy, increment accumulator to ", self.accumulator)
        else:
            self.accumulator = 0
        self.val_accuracy = current

In [13]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    #array = cv2.cvtColor(array, cv2.COLOR_BGR2GRAY)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [9]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

In [35]:
train_open_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\train\Open'
test_open_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\test\Open'
train_closed_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\train\Closed'
test_closed_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\test\Closed'

change_size(train_open_root)
change_size(test_open_root)
change_size(train_closed_root)
change_size(test_closed_root)

Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\train\Open\rescaled 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\test\Open\rescaled 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\train\Closed\rescaled 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\test\Closed\rescaled 


In [36]:
train_open_root_rescaled = os.path.join(train_open_root,"rescaled")
test_open_root_rescaled = os.path.join(test_open_root,"rescaled")
train_closed_root_rescaled = os.path.join(train_closed_root,"rescaled")
test_closed_root_rescaled = os.path.join(test_closed_root,"rescaled")

paths = [train_open_root_rescaled,test_open_root_rescaled,train_closed_root_rescaled,test_closed_root_rescaled]

for path in paths:
    length = len(os.listdir(path))
    augmentor_train_pipeline = aug.Pipeline(path)
    augmentor_train_pipeline.rotate(probability=0.8, max_left_rotation=10, max_right_rotation=10)
    augmentor_train_pipeline.skew_tilt(probability=0.75,magnitude = 0.6)
    augmentor_train_pipeline.random_distortion(probability=0.75, grid_width= 50, grid_height = 50, magnitude = 2)
    augmentor_train_pipeline.flip_left_right(probability=0.75)
    augmentor_train_pipeline.zoom(probability=0.75, min_factor=1.1, max_factor=1.3)
    augmentor_train_pipeline.status()
    augmentor_train_pipeline.sample(length*10)

final_train_open_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Open"
final_train_closed_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Closed"
final_test_open_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Open"
final_test_closed_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed"

create_directory(final_train_open_path)
create_directory(final_train_closed_path)
create_directory(final_test_open_path)
create_directory(final_test_closed_path)

for files in os.listdir(os.path.join(train_open_root_rescaled,"Output")):
    source = os.path.join(os.path.join(train_open_root_rescaled,"Output"),files)
    destination = os.path.join(final_train_open_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_train_open_path,name1)
    dest2 = os.path.join(final_train_open_path,name2)
    dest3 = os.path.join(final_train_open_path,name2)
    dest4 = os.path.join(final_train_open_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(train_open_root_rescaled)

for files in os.listdir(os.path.join(test_open_root_rescaled,"Output")):
    source = os.path.join(os.path.join(test_open_root_rescaled,"Output"),files)
    destination = os.path.join(final_test_open_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_test_open_path,name1)
    dest2 = os.path.join(final_test_open_path,name2)
    dest3 = os.path.join(final_test_open_path,name2)
    dest4 = os.path.join(final_test_open_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(test_open_root_rescaled)

for files in os.listdir(os.path.join(train_closed_root_rescaled,"Output")):
    source = os.path.join(os.path.join(train_closed_root_rescaled,"Output"),files)
    destination = os.path.join(final_train_closed_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_train_closed_path,name1)
    dest2 = os.path.join(final_train_closed_path,name2)
    dest3 = os.path.join(final_train_closed_path,name2)
    dest4 = os.path.join(final_train_closed_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(train_closed_root_rescaled)
    
for files in os.listdir(os.path.join(test_closed_root_rescaled,"Output")):
    source = os.path.join(os.path.join(test_closed_root_rescaled,"Output"),files)
    destination = os.path.join(final_test_closed_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_test_closed_path,name1)
    dest2 = os.path.join(final_test_closed_path,name2)
    dest3 = os.path.join(final_test_closed_path,name2)
    dest4 = os.path.join(final_test_closed_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(test_closed_root_rescaled)

Initialised with 2295 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\train\Open\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 2295
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 2
	 JPEG
	 PNG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=L size=80x80 at 0x1570E7C0EB0>: 100%|█| 22950/22950 [16:37<00:00, 23.00 Samples/


Initialised with 449 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\test\Open\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 449
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 2
	 JPEG
	 PNG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=L size=80x80 at 0x15709FDE970>: 100%|█| 4490/4490 [03:18<00:00, 22.64 Samples/s]


Initialised with 2295 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\train\Closed\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 2295
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 2
	 JPEG
	 PNG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=L size=80x80 at 0x1570E60B280>: 100%|█| 22950/22950 [17:22<00:00, 22.02 Samples/


Initialised with 448 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data - copia\test\Closed\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 448
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 2
	 JPEG
	 PNG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=L size=80x80 at 0x15709AFCD90>: 100%|█| 4480/4480 [03:14<00:00, 23.02 Samples/s]


Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Open 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Closed 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Open 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed 


In [43]:
train_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train'
test_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test'
x_train, y_train, train_names = load_images(train_root)
x_test, y_test, test_names = load_images(test_root)

In [44]:
x_train.shape

(137700, 80, 80, 1)

In [52]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu',padding='same', input_shape=(80, 80, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 128)       7

In [53]:
change_lr_callback = CustomCallbackReduceLr2()
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model/eye_classifier1_v4.h5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
history = model.fit(x_train, y_train, batch_size=50, epochs=100, verbose=1, validation_split=0.2, callbacks = [change_lr_callback, model_checkpoint_callback])

Epoch 1/100
2204/2204 [==============================] - 308s 139ms/step - loss: 0.0985 - accuracy: 0.9629 - val_loss: 0.0158 - val_accuracy: 0.9961
No change on val accuracy, increment accumulator to  1
Epoch 2/100
2204/2204 [==============================] - 306s 139ms/step - loss: 0.0153 - accuracy: 0.9950 - val_loss: 0.0053 - val_accuracy: 0.9985
No change on val accuracy, increment accumulator to  2
Epoch 3/100
2204/2204 [==============================] - 302s 137ms/step - loss: 0.0089 - accuracy: 0.9971 - val_loss: 0.0013 - val_accuracy: 0.9995
No change on val accuracy, increment accumulator to  3
Epoch 4/100
2204/2204 [==============================] - 303s 137ms/step - loss: 0.0059 - accuracy: 0.9982 - val_loss: 0.0021 - val_accuracy: 0.9992
No change on val accuracy, increment accumulator to  4
Epoch 5/100
2204/2204 [==============================] - 302s 137ms/step - loss: 0.0053 - accuracy: 0.9983 - val_loss: 0.0053 - val_accuracy: 0.9980
No change on val accuracy, incremen

KeyboardInterrupt: 

In [54]:
y_predicted = []
start_time = time.time()
for i in range(len(y_test)):
    out_probabilities = model.predict(x_test[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    #if number_detected != y_test[i]:
    #    print(test_names[i])
    #    print(out_probabilities[0][0])
end_time = time.time()
mean = (end_time - start_time) / len(y_test)
a = model.evaluate(x_test, y_test)
print(classification_report(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted))
print("Time for prediction: {}".format(mean))

841/841 [==============================] - 24s 29ms/step - loss: 0.0354 - accuracy: 0.9973
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13440
           1       1.00      1.00      1.00     13470

    accuracy                           1.00     26910
   macro avg       1.00      1.00      1.00     26910
weighted avg       1.00      1.00      1.00     26910

[[13392    48]
 [   25 13445]]
Time for prediction: 0.04482897045620309


In [ ]:
y_predicted = []
start_time = time.time()
for i in range(len(y_train)):
    out_probabilities = model.predict(x_train[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    if number_detected != y_train[i]:
        print(train_names[i])
end_time = time.time()
mean = (end_time - start_time) / len(y_train)
a = model.evaluate(x_train, y_train)
print(classification_report(y_train, y_predicted))
print(confusion_matrix(y_train, y_predicted))
print("Time for prediction: {}".format(mean))

In [39]:
pruebas_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas'
x_pruebas, y_pruebas, pruebas_names = load_images(pruebas_root)

In [41]:
y_predicted = []
start_time = time.time()
for i in range(len(y_pruebas)):
    out_probabilities = model.predict(x_pruebas[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    #if number_detected != y_pruebas[i]:
        #print(pruebas_names[i])
        #print(out_probabilities[0][0])
end_time = time.time()
mean = (end_time - start_time) / len(y_pruebas)
a = model.evaluate(x_pruebas, y_pruebas)
print(classification_report(y_pruebas, y_predicted))
print(confusion_matrix(y_pruebas, y_predicted))
print("Time for prediction: {}".format(mean))

125/125 [==============================] - 3s 24ms/step - loss: 0.7013 - accuracy: 0.9865
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      2000
           1       1.00      0.97      0.99      2000

    accuracy                           0.99      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000

[[2000    0]
 [  54 1946]]
Time for prediction: 0.051617367506027224


In [37]:
test2_open_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\video\Open'
test2_close_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\video\Close'
change_size(test2_open_root)
change_size(test2_close_root)

Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\video\Open\rescaled 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\video\Close\rescaled 


In [38]:
root2 = r'C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\test'

x_prueba2, y_prueba2, prueba2_names = load_images(root2)
y_predicted = []
for i in range(len(y_prueba2)):
    out_probabilities = model.predict(x_prueba2[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.05 else 0
    y_predicted.append(number_detected)
    if number_detected != y_prueba2[i]:
        print(prueba2_names[i])
a = model.evaluate(x_prueba2, y_prueba2)
print(classification_report(y_prueba2, y_predicted))
print(confusion_matrix(y_prueba2, y_predicted))

C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\test\closed\eye_180.jpg
C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\test\closed\eye_mauricio198.jpg
C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\test\open\eye_mauricio306.jpg
22/22 [==============================] - 1s 21ms/step - loss: 0.0449 - accuracy: 0.9943
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       325
           1       0.99      1.00      1.00       372

    accuracy                           1.00       697
   macro avg       1.00      1.00      1.00       697
weighted avg       1.00      1.00      1.00       697

[[323   2]
 [  1 371]]


In [78]:
model.save("model/eye_classifier1_v4.h5")
#model2.save("model/eye_classifier2_v2.h5")
print("Saved model to disk")

Saved model to disk
